https://github.com/prdeepakbabu/Python/blob/master/Deep%20learning%20gender/Deep%20Learning%20(RNN%20-%20LSTMs)%20Predict%20Gender%20from%20Name.ipynb 


In [1]:
import tensorflow as tf
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding
from keras.layers import LSTM, Embedding
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

In [2]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from sklearn.metrics import classification_report

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"

In [5]:
%cd $PATH_TO_FOLDER

/content/drive/MyDrive/Name2Demographics


In [6]:
import sys  
sys.path.insert(0, 'Models/PreProcessing/')

from char_utils import *

sys.path.insert(0, 'Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

In [7]:
cbse_df = CBSEData()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (84,88,90,94,96,100,102,106,108,112,114,118,120,123,125,128,130,134,136,140,142,145,147,151,153,157,159,163,165,169,171,175,177,180,182,185,187,191,193,194,196,200,202,206,208,211,213,216,218,222,224,228,230,234,236,240,242,246,248,252,254,257,259,263,265,269,271,275,277,281,283,287,289,293,295,299,301,305,307,311,313,317,319,323,325,328,330,334,336,340,342,346,348,351,353,354,356,359,361,364,366,370,372,376,378,381,383,387,389,393,395,399,401,405,407,411,413,417,419,423) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (82,86,88,92,94,98,100,104,106,110,112,116,124,128,130,134,136,140,142,146,148,152,154,158,160,164,166,170,172,176,178,182,184,188,190,194,196,200,202,206,208,212,214,218,220,224,226,230,232,233,235,239,241,245,247,2

In [8]:
train_x, train_y, val_x, val_y, test_x, test_y = splitData(cbse_df, 0.7, 0, 'Name', 'char_cnn_cbse_oov.txt')

(train:(151291, 4), val:(0, 0), test:(53253, 4))
Train: ((151240, 30),(151240, 2)) Val:((0,),(0,)) Test: ((53239, 30),(53239, 2))


In [9]:
voc = vocab()

#### https://stackoverflow.com/questions/59265920/this-tensorflow-binary-is-optimized-with-intelr-mkl-dnn-to-use-the-following-c


In [ ]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

In [13]:
inputs = Input(MAXLEN_NAME_CHAR_CNN)
x = Embedding(len(voc), len(voc))(inputs)
x = Conv1D(filters = 256, kernel_size = 7, activation = 'relu')(x)
x = Flatten()(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 embedding (Embedding)       (None, 30, 28)            784       
                                                                 
 conv1d (Conv1D)             (None, 24, 256)           50432     
                                                                 
 flatten (Flatten)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 2)                 12290     
                                                                 
Total params: 63,506
Trainable params: 63,506
Non-trainable params: 0
_________________________________________________________________


In [16]:
batch_size=1000
model.fit(train_x, train_y,batch_size=batch_size,epochs=15,validation_data=(test_x, test_y))

Epoch 1/15
152/152 [==============================] - 13s 22ms/step - loss: 0.3731 - accuracy: 0.8240 - val_loss: 0.2693 - val_accuracy: 0.8916
Epoch 2/15
152/152 [==============================] - 3s 20ms/step - loss: 0.2039 - accuracy: 0.9241 - val_loss: 0.2272 - val_accuracy: 0.9097
Epoch 3/15
152/152 [==============================] - 3s 20ms/step - loss: 0.1641 - accuracy: 0.9413 - val_loss: 0.2156 - val_accuracy: 0.9110
Epoch 4/15
152/152 [==============================] - 3s 22ms/step - loss: 0.1415 - accuracy: 0.9495 - val_loss: 0.2062 - val_accuracy: 0.9169
Epoch 5/15
152/152 [==============================] - 3s 19ms/step - loss: 0.1269 - accuracy: 0.9551 - val_loss: 0.1977 - val_accuracy: 0.9192
Epoch 6/15
152/152 [==============================] - 3s 19ms/step - loss: 0.1160 - accuracy: 0.9595 - val_loss: 0.1922 - val_accuracy: 0.9226
Epoch 7/15
152/152 [==============================] - 3s 22ms/step - loss: 0.1076 - accuracy: 0.9625 - val_loss: 0.1973 - val_accuracy: 0.923

In [17]:
score, acc = model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

1664/1664 [==============================] - 6s 4ms/step - loss: 0.1884 - accuracy: 0.9298
Test score: 0.18836355209350586
Test accuracy: 0.9297695159912109


In [18]:
pred = model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male     0.9543    0.9129    0.9331     28576
      Female     0.9039    0.9494    0.9261     24663

    accuracy                         0.9298     53239
   macro avg     0.9291    0.9311    0.9296     53239
weighted avg     0.9309    0.9298    0.9299     53239

--------------------------------------------------------


In [19]:
filename = "Models/CBSEData/CharCNN/SavedModel/CharCNN"
model.save(filename)

INFO:tensorflow:Assets written to: Models/CBSEData/CharCNN/SavedModel/CharCNN/assets
